## 1. Create a Flask app that displays the most recent articles from Wired's top stories RSS feed (https://www.wired.com/feed/rss) along with their summary and pub-time. 

1. `mkdir wired_app`
2. `cd wired_app`
3. `touch wired.py`
4. `mkdir templates`
5. `cd templates`
6. `touch wired.html`

#### `wired.py`

```python 
from flask import Flask, render_template
import feedparser
import random
import time

app = Flask(__name__)

@app.route('/')
def home():

    wired_url = "https://www.wired.com/feed/rss"
    wired_feed = feedparser.parse(wired_url)

    article_data = [{"title": wired_feed['entries'][i]['title'], "summary": wired_feed['entries'][i]['summary'], "published": wired_feed['entries'][i]['published']} for i in range(0,len(wired_feed))]

    return render_template("wired.html", article_data=article_data)

app.run(host='0.0.0.0', port=5000, debug=True) # anyone can connect, and we're running on port 5000
```
#### `wired.html`
```html
<!DOCTYPE html>
<html>
<body>
  <table>
    <thead>
      <tr>
        <th>Title</th>
        <th>Summary</th>
        <th>Published</th>
      </tr>
    </thead>
    <tbody>
      {% for i in article_data %}
    <tr>
        <td>{{ i['title'] }}</td>
        <td>{{ i['summary'] }}</td>
        <td>{{ i['published'] }}</td>
    </tr>
      {% endfor %}
    </tbody>
  </table>
</body>
</html>

```


## 2. Use the same Bootstrap elements as we did in class to prettify your table of published stories. 

#### `wired.html` 

```html 
<!DOCTYPE html>
<html>
<head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css" integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO" crossorigin="anonymous">
</head>
<body>
    <div class="container">
        <div class="panel panel-info">
            <div class="panel-heading">
                <h1 class="panel-title">Wired Info</h1>
            </div>
            <table  class="table table-striped table-bordered table-hover">
    <thead>
      <tr>
        <th>Title</th>
        <th>Summary</th>
        <th>Published</th>
      </tr>
    </thead>
    <tbody>
      {% for i in article_data %}
	<tr>
        <td>{{ i.title }}</td>
        <td>{{ i.summary }}</td>
        <td>{{ i.published }}</td>
	</tr>
      {% endfor %}
    </tbody>
    </table>
    <img class="center-block" src="{{image}}" />
  </div>
</div>
</body>
</html>
``` 

## 3. If a Wired story was published in the last hour, display that title in red; otherwise, in blue.

#### `wired.py` 
```python 
from flask import Flask, render_template
import feedparser
import random
import time
from datetime import datetime, timezone
from datetime import timedelta
import pandas as pd

app = Flask(__name__)

@app.route('/')
def home():

    wired_url = "https://www.wired.com/feed/rss"
    wired_feed = feedparser.parse(wired_url)

    article_data = [{"title": wired_feed['entries'][i]['title'], "summary": wired_feed['entries'][i]['summary'], "published": pd.to_datetime(wired_feed['entries'][i]['published'])} for i in range(0,len(wired_feed))]

    now = datetime.now(timezone.utc)

    hours = now + timedelta(hours=-1)

    return render_template("wired.html", article_data=article_data, now=now, hours=hours)

app.run(host='0.0.0.0', port=5000, debug=True) # anyone can connect, and we're running on port 5000

```

#### `wired.html` 
```html 
<!DOCTYPE html>
<html>
<body>
  <table>
    <thead>
      <tr>
        <th>Title</th>
        <th>Summary</th>
        <th>Published</th>
      </tr>
    </thead>
    <tbody>
      {% for i in article_data %}
      <tr>
        {% if i['published'] <= hours  %}
            <td style = "color: red">{{ i['title'] }}</td>
            <td style = "color: red">{{ i['summary'] }}</td>
            <td style = "color: red">{{ i['published'] }}</td>
        {% endif %}
        {% if i['published'] >= hours  %}
            <td style = "color: blue">{{ i['title'] }}</td>
            <td style = "color: blue">{{ i['summary'] }}</td>
            <td style = "color: blue">{{ i['published'] }}</td>
        {% endif %}
      </tr>
      {% endfor %}
    </tbody>
  </table>
</body>
</html>
```

## 4. Create a histogram of average title length (in characters) and display it via your Flask app.

#### `wired.py`
``` python
from flask import Flask, render_template
import feedparser
import random
import time
from datetime import datetime, timezone
from datetime import timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import *

matplotlib.use('Agg')

app = Flask(__name__)

def create_plot():


    wired_url = "https://www.wired.com/feed/rss"
    wired_feed = feedparser.parse(wired_url)

    article_data = [{"title": wired_feed['entries'][i]['title'], "summary": wired_feed['entries'][i]['summary'], "published": pd.to_datetime(wired_feed['entries'][i]['published'])} for i in range(0,len(wired_feed))]

    title_length_list = []

    for i in article_data:
        title = i['title']
        title_length = len(title)
        title_length_list.append(title_length)

    df = pd.DataFrame(title_length_list)

    fig = plt.hist(title_length_list, bins=5)

    plt.title('Wired Article Title Length')
    plt.xlabel("Length (In Characters)")
    plt.ylabel("Frequency")
    plt.savefig("abc.png")

    filename = 'static/plot-'+str(datetime.now(timezone.utc))+ '.png'
    plt.savefig(filename)

    return filename

@app.route('/')
def home():

    wired_url = "https://www.wired.com/feed/rss"
    wired_feed = feedparser.parse(wired_url)

    article_data = [{"title": wired_feed['entries'][i]['title'], "summary": wired_feed['entries'][i]['summary'], "published": pd.to_datetime(wired_feed['entries'][i]['published'])} for i in range(0,len(wired_feed))]

    now = datetime.now(timezone.utc)

    hours = now + timedelta(hours=-1)

    image_filename = create_plot()

    return render_template("wired.html", article_data=article_data, now=now, hours=hours, image=image_filename)

app.run(host='0.0.0.0', port=5000, debug=True) # anyone can connect, and we're running on port 5000
```
#### `wired.html`
```html
<!DOCTYPE html>
<html>
<body>
  <table>
    <thead>
      <tr>
        <th>Title</th>
        <th>Summary</th>
        <th>Published</th>
      </tr>
    </thead>
    <tbody>
      {% for i in article_data %}
      <tr>
        {% if i['published'] <= hours  %}
            <td style = "color: red">{{ i['title'] }}</td>
            <td style = "color: red">{{ i['summary'] }}</td>
            <td style = "color: red">{{ i['published'] }}</td>
        {% endif %}
        {% if i['published'] >= hours  %}
            <td style = "color: blue">{{ i['title'] }}</td>
            <td style = "color: blue">{{ i['summary'] }}</td>
            <td style = "color: blue">{{ i['published'] }}</td>
        {% endif %}
      </tr>
      {% endfor %}
    </tbody>
  </table>
  <img class="center-block" src="{{image}}" />
</body>
</html>
```